In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
from h3 import h3
import os
from os.path import isfile, join
from pathlib import Path
from os import listdir

path =  Path(os.getcwd())
root = path.parent.absolute()

import warnings
pd.options.mode.chained_assignment = None  # default='warn'

h3_level = 8

In [ ]:
crash_dir = root / 'X.data' / 'raw_data' / 'TIMS_raw_crashes_downloads'

onlyfiles = [f for f in listdir(crash_dir) if isfile(join(crash_dir, f))]

appended_data = []

for file in onlyfiles:
    print(file)
    temp = pd.read_csv(crash_dir / file, low_memory = False)
    appended_data.append(temp)

collision_df = pd.concat(appended_data)
collision_df.sample(2)

In [ ]:
collision_df['COLLISION_TIME']=collision_df['COLLISION_TIME'].apply(lambda x: '{0:0>4}'.format(x))

mask = collision_df['COLLISION_TIME']=='2500'
valid_time = collision_df[~mask]
nonvalid_time = collision_df[mask]
nonvalid_time['COLLISION_HOUR'] = 'none'
print(nonvalid_time.shape)
#350 records.  going to drop

valid_time = valid_time.assign(COLLISION_HOUR=pd.to_datetime(valid_time.COLLISION_TIME, format='%H%M').dt.hour)
valid_time = valid_time.assign(COLLISION_DATE=pd.to_datetime(valid_time.COLLISION_DATE, format='%Y-%m-%d'))

valid_time['COLLISION_YEAR'] = valid_time.COLLISION_DATE.dt.year
valid_time['COLLISION_MONTH'] = valid_time.COLLISION_DATE.dt.month
valid_time['COLLISION_DAY'] = valid_time.COLLISION_DATE.dt.day
valid_time['COLLISION_DAYOFWEEK'] = valid_time.COLLISION_DATE.dt.dayofweek
valid_time['COLLISION_DAYOFYEAR'] = valid_time.COLLISION_DATE.dt.dayofyear

valid_time.sample(2)

In [ ]:
valid_time.columns = [each_string.lower() for each_string in valid_time.columns]

In [ ]:
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row.point_y, row.point_x, h3_level)

pd_dict = {}

valid_time['hex_id'] = valid_time.apply(lat_lng_to_h3, axis=1)
print(valid_time.shape)
valid_time = valid_time.groupby(['hex_id', 'collision_year', 'collision_dayofyear', 'collision_month', 'collision_dayofweek', 'collision_hour']).first()
valid_time.reset_index(inplace = True)

valid_time_select = valid_time[['hex_id', 'collision_year', 'collision_dayofyear', 'collision_month', 'collision_dayofweek', 'collision_hour']]
valid_time_select['accident_count'] = 1
print(valid_time_select.shape)
#valid_time_cnt.sample(3)
#valid_time_cnt.columns = ['h3_'+str(level), 'collision_count_h3_'+str(level)]
#pd_dict[key_val] = valid_time_cnt

In [ ]:
valid_time_select.to_csv(root / 'X.data' / 'h3_processed_data' / 'collisions_hex.csv', index=False)

valid_time.to_csv(root / 'X.data' / 'all_collisions_points.csv', index=False)